### Intrinsic Valuation of Stocks Using Python
* **Link:** https://medium.com/swlh/intrinsic-valuation-of-stocks-using-python-5d902a34b1a0
* **Summary/findings:** Purpose of this article is to help create a discounted cash flow model in python to calculate intrinsic value of a company. This helps to see whether or not a company is overvalued which can give help information for trading. Not exactly a trading strategy but good exploration of things that are impactful for stock market. 
* **Code: Evaluation.**
* * **File:** https://github.com/shreyawalia259/tradingstrategies/blob/main/Implemented%20Examples/Evaluation.ipynb
<h3>

Importing important libraries to get the Discounted Cash Flow (DCF) of a company. 

In [8]:
import numpy as np
import pandas as pd

Populating our sales series from 2019 to 2024. Since we only use 2019 data, we enter that data and work towards our goal of adding sales data for following years using growth rate for that year.

In [9]:
years=['2019A', '2020F', '2021F', '2022F', '2023F', '2024F']
sales = pd.Series(index=years)
sales['2019A'] = 15
sales

<ipython-input-9-8fd422aacd4c>:2: DeprecationWarning: The default dtype for empty Series will be 'object' instead of 'float64' in a future version. Specify a dtype explicitly to silence this warning.
  sales = pd.Series(index=years)


2019A    15.0
2020F     NaN
2021F     NaN
2022F     NaN
2023F     NaN
2024F     NaN
dtype: float64

We set the growth rate to be 0.1

In [10]:
growth_rate = 0.1

Next, we create a loop to populate the following for each year->
* Sales
* EBIT- Operating Income
* Tax Expense
* NOPAT
* D&A Expense
* Capital Expenditures
* Increase in NWC
* Free Cash Flow

These will help us calculate the DCF. 

First we add the sales, using sales from previous year * growth rate

In [11]:
for year in range(1,6):
    sales[year] = sales[year-1] * (1+growth_rate)

Next, we work on getting the operating income. Using the ebit margin and depreciation rate, we can get calculate the operating income. These will have to be changed according to company that is being evaluated. 

In [12]:
    ebitda_margin = 0.20
    depr_percent = 0.03
    ebitda = sales * ebitda_margin
    depreciation = sales * depr_percent
    ebit = ebitda - depreciation

Then, we will calculate tax payments. Using the tax rate, we can get tax payment by taxing operating income and factoring in the tax rate on that income. 

In [13]:

    tax_rate = 0.30
    tax_payment = -ebit * tax_rate
    tax_payment = tax_payment.apply(lambda x: min(x,0))

We can then get the Net Operating Income after Taxes(NOPAT) by taking our operating income and adding our tax payment to it.

In [14]:
    nopat = ebit + tax_payment

Next, we will working on getting the Net Working Capital(NWC). 
* For the purpose of this evaluation, we only care about the change in NWC.
So we calculate the change by taking current NWC as a percent of sales and then taking the difference from the previous year. 

In [15]:
    nwc_percent = 0.24
    nwc = sales * nwc_percent
    change_in_nwc = nwc.shift(1) - nwc

* Now get the Capital Expenditures (Capex) by taking a negative value since it is an expense and then taking the depreciation percent from sales. 
* We do this because in the long run, companies capital expenditures should equal the depreciation expense. 

In [16]:
    capex_percent = depr_percent
    capex = -(sales * capex_percent)

Finally we calculate our free cash flow before we can get the DCF.

In [17]:
    free_cash_flow = nopat + depreciation + capex + change_in_nwc

**Getting the DCF**
A DCF has a near-future component and a long-term future component. The near-future part is the 5 years that we have calculated above, and the long-term part is perpetuity calculated using the 5th forecasted year’s FCF as a base into the future. This larger lump sum is called the Terminal Value.

**Terminal Value**
Terminal value is calculated using the last year’s FCF forecast after applying the terminal growth rate of 2%, essentially giving us the 6th year’s FCF. We apply the discount factor, a number ranging from 0 to 1, to each of the numbers, and summing the 5 year PV FCF amounts with the PV of the terminal value


**Note**: DCF Value is returned in millions

In [18]:
    cost_of_capital = 0.10
    terminal_growth = 0.02
    terminal_value = ((free_cash_flow[-1] * (1 + terminal_growth))/
	                 (cost_of_capital - terminal_growth))
    discount_factors = [(1 / (1 + cost_of_capital)) ** i for i in range (1,6)]
    dcf_value = (sum(free_cash_flow[1:]*discount_factors) +
	            terminal_value * discount_factors[-1])
    dcf_value

25.874659090909088

Lastly, we can export the dataframe of the values used to calculate DCF into excel

In [19]:
output = pd.DataFrame([sales, ebit, tax_payment, nopat, 
                       depreciation, capex, change_in_nwc,
                       free_cash_flow],
                     index=["Sales", "EBIT", "Tax Expense", 
                            "NOPAT", "D&A Expense",
                            "Capital Expenditures",
                            "Increase in NWC",
                            "Free Cash Flow"]).round(2)
print(output)

                      2019A  2020F  2021F  2022F  2023F  2024F
Sales                 15.00  16.50  18.15  19.97  21.96  24.16
EBIT                   2.55   2.80   3.09   3.39   3.73   4.11
Tax Expense           -0.76  -0.84  -0.93  -1.02  -1.12  -1.23
NOPAT                  1.78   1.96   2.16   2.38   2.61   2.87
D&A Expense            0.45   0.50   0.54   0.60   0.66   0.72
Capital Expenditures  -0.45  -0.50  -0.54  -0.60  -0.66  -0.72
Increase in NWC         NaN  -0.36  -0.40  -0.44  -0.48  -0.53
Free Cash Flow          NaN   1.60   1.76   1.94   2.13   2.35
